In [8]:
## All required packages
## Scientific Packakges
import sys, os, re, gc, time
import numpy as np
import pandas as pd
from random import sample
from scipy.io import savemat
from collections import Counter
import matplotlib.pyplot as plt

## Keras
from keras import metrics
from keras import optimizers
from keras.utils.np_utils import to_categorical
from keras.models import Model, load_model
from keras.layers import Input, Dense, BatchNormalization, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint

## Sklearn-metrics
from sklearn.metrics import confusion_matrix, accuracy_score, matthews_corrcoef, balanced_accuracy_score, roc_auc_score

## Custom functions
from models import E3_LSE
from utils import *
from feature_encodings import * ## Feature encoding utils
from perf_metrics import * ## Some of hand crafted performance metrics

%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
## Loading and pre-processing E3-Target dataset
data_path = 'https://raw.githubusercontent.com/psychemistz/Colab_temp/master/E3target_pred/1metadata.csv'
dataset = pd.read_csv(data_path, index_col=None)

In [3]:
## Extract features
E3_features, Sub_features, Pair_features = Extract_Features(dataset, gap_size=6)
target_label = to_categorical(dataset['Label'])

In [6]:
# def MCSim(LVs, Gaps, lmd, num_Trials, E3_features, Sub_features, Pair_features, target_label):
#     ##### Perform Monte-Carlos Simulations for [num_Trials]# of independent Trials####
    
#     ## Define parameters explicitly
#     start = time.time()
#     LVs = LVs
#     Gaps = Gaps
#     lmd = lmd
#     num_Trials = num_Trials
    
#     print("start MC Simulation")
    
#     ## Start loops    
#     for gap in Gaps:
#         for LV in LVs:
#             Stats =[]            
#             ## Divide negative and positive samples
#             plist = list(np.asarray(np.where(target_label[:,1]==1)).flatten())
#             nlist = list(np.asarray(np.where(target_label[:,1]==0)).flatten())
            
#             for loop_ind in range(0,num_Trials):
#                 ## Split Datasets
#                 train_list, val_list, test_list = split_dataset(plist, nlist, target_label, num_train=570, num_valid=30)
                
#                 ## Load Datasets
#                 Xin_train, Xout_train, y_train = load_dataset(E3_features, Sub_features, Pair_features, target_label, train_list)
#                 Xin_val, Xout_val, y_val = load_dataset(E3_features, Sub_features, Pair_features, target_label, val_list)
#                 Xin_test, Xout_test, y_test = load_dataset(E3_features, Sub_features, Pair_features, target_label, test_list)
                
#                 # Define Model
#                 model = E3_LSE(input_size=Xin_train.shape[1],output_size=Xout_train.shape[1],LV=LV, lmd=lmd)
#                 es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=300)
                
#                 checkpoint = ModelCheckpoint('mc_model-best.h5',
#                                              verbose=0, monitor='val_loss',save_best_only=True, mode='auto')
                
#                 history = model.fit({'enc_input': Xin_train},
#                                     {'class_output': y_train, 'decoder_output': Xout_train},
#                                     validation_data = ({'enc_input': Xin_val},
#                                                        {'class_output': y_val, 'decoder_output': Xout_val}),
#                                     epochs=2000, batch_size=1000, callbacks=[checkpoint, es], verbose=0)
                
#                 del model  # deletes the existing model
#                 model = load_model('mc_model-best.h5')
                
#                 ## Calculate Outputs
#                 y_train_pred, X_train_pred = model.predict(Xin_train,batch_size=1800, verbose=0)
#                 y_train_pred = to_categorical(y_train_pred.argmax(axis=1))
#                 MSE_X_train_pred = (np.square(X_train_pred - Xout_train)).mean(axis=1)
                
#                 y_val_pred, X_val_pred = model.predict(Xin_val,batch_size=200, verbose=0)
#                 y_val_pred = to_categorical(y_val_pred.argmax(axis=1))
#                 MSE_X_val_pred = (np.square(X_val_pred - Xout_val)).mean(axis=1)
                
#                 y_test_pred, X_test_pred = model.predict(Xin_test,batch_size=200, verbose=0)
#                 y_test_pred = to_categorical(y_test_pred.argmax(axis=1))
#                 MSE_X_test_pred = (np.square(X_test_pred - Xout_test)).mean(axis=1)
                
#                 ## Performance Measures
#                 tr_acc, tr_sen, tr_spe, tr_f1, tr_mcc, tr_bacc, tr_yi, tr_auc = Calculate_Stats(y_train,y_train_pred);
#                 v_acc, v_sen, v_spe, v_f1, v_mcc, v_bacc, v_yi, v_auc = Calculate_Stats(y_val,y_val_pred);
#                 t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, t_auc = Calculate_Stats(y_test,y_test_pred);

#                 ## Save Measures for later analysis
#                 Stats.append([tr_acc, tr_sen, tr_spe, tr_f1, tr_mcc, tr_bacc, tr_yi, tr_auc, -10*np.log10(MSE_X_train_pred.mean()),
#                               v_acc, v_sen, v_spe, v_f1, v_mcc, v_bacc, v_yi, v_auc, -10*np.log10(MSE_X_val_pred.mean()),
#                               t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, t_auc, -10*np.log10(MSE_X_test_pred.mean())])
                
#                 ## Print performance messages
#                 print('CKSAAP-Gap:',gap, 'LV=',LV,'Trial:',loop_ind, 'Test Youden-index:', t_yi, 'MCC:', t_mcc, 'AUC:', t_auc, 'MSE (dB):', -10*np.log10(MSE_X_test_pred.mean()))
#                 ## End of single trial
            
#             ## save all trials
#             Statistics = np.asarray(Stats)
#             filename = 'E3_LSE_STATS_CKSAAP_GAP_' + str(gap) + 'LV' + str(LV) + 'cls' + str(0.99) +'.mat'
#             savemat(filename,{'Statistics':Statistics})
                
#         ## Show Classification/Reconstruction Statistics for given LV and gap
#         Show_Statistics('Training Results (MEAN)',Statistics.mean(axis=0)[0:9])
#         Show_Statistics('Validation Results (MEAN)',Statistics.mean(axis=0)[9:18])
#         Show_Statistics('Test Results (MEAN)',Statistics.mean(axis=0)[18:27])
            
#     end = time.time()
#     print(end-start)

In [9]:
"""LVs = Number of Latent Variables
   Gaps = Number of Gaps in CKSAAP Feature
   lmd = Weight of Reconstruction Loss (Lambda)
   num_Trials = Number of Trials for each LV, Gap combination at given Lambda value. 
"""
LVs = np.asarray(range(2,6))
Gaps = np.asarray(range(0,9))
lmd = 0.5 
num_Trials = 1

MCSim(LVs, Gaps, lmd, num_Trials, E3_features, Sub_features, Pair_features, target_label)

CKSAAP-Gap: 0 LV= 2 Trial: 0 Test Youden-index: 0.4566707953640625 MCC: 0.32896427251312305 AUC: 0.7283353976820313 MSE (dB): 21.17294480866242


KeyboardInterrupt: 